In [33]:
import os
import pandas as pd
from datetime import datetime
org_path = os.getcwd()
data_path = org_path + '/stock-data'
data_csv = [csv for csv in os.listdir(data_path) if csv.endswith('.csv')]
print (data_csv)

['0005_trade_20161001-20170112.csv', '0494_trade_20161001-20170112.csv', '0700_trade_20161001-20170112.csv', '0939_trade_20161001-20170112.csv', '0941_trade_20161001-20170112.csv', '1299_trade_20161001-20170112.csv', '2318_trade_20161001-20170112.csv', '2333_trade_20161001-20170112.csv', '2628_trade_20161001-20170112.csv']


In [34]:
dt_0005 = pd.read_csv('%s/0005_trade_20161001-20170112.csv' % (data_path))
dt_0005['Date'] = pd.to_datetime(dt_0005['UpdatedTime'], format="%Y-%m-%d").dt.date
dt_0005['Time'] = pd.to_datetime(dt_0005['UpdatedTime'],format= '%Y-%m-%d %H:%M:%S.%f').dt.time
cols = dt_0005.columns.tolist()
cols = cols[-2:] + cols[:-2]
dt_0005 = dt_0005[cols]
del dt_0005['UpdatedTime']
del dt_0005['Side']
print (dt_0005.head())
print (dt_0005.shape)
opening = datetime.strptime('09:30:00', '%H:%M:%S').time()
dt_0005 = dt_0005.loc[dt_0005['Time'] >= opening]
closing = datetime.strptime('16:00:00', '%H:%M:%S').time()
dt_0005 = dt_0005.loc[dt_0005['Time'] <= closing]
date = list(set(dt_0005['Date'].tolist()))
date = sorted(date)
print ('number of date: %d days'%(len(date)))
print (date[0])

         Date      Time  Symbol  Price  Quantity
0  2016-10-03  09:20:00       5  58.15      2400
1  2016-10-03  09:20:00       5  58.15      5200
2  2016-10-03  09:20:00       5  58.15      1200
3  2016-10-03  09:20:00       5  58.15       400
4  2016-10-03  09:20:00       5  58.15       800
(549785, 5)
number of date: 67 days
2016-10-03


In [35]:
def splitting_dt(dt_summary, dt_stock, date, next_date):
    dt_stock_next = dt_stock.loc[dt_stock['Date'] == next_date]
    next_day_change = dt_stock_next['Price'].iloc[0] - dt_stock['Price'].iloc[-1]
    
    dt_stock = dt_stock.loc[dt_stock['Date'] == date] # sorting the date
    last_five = datetime.strptime('15:45:00', '%H:%M:%S').time() # deleting the last 15 mins    
    dt_stock = dt_stock.loc[dt_stock['Time'] <= last_five]
    high = dt_stock['Price'].max()
    low =  dt_stock['Price'].min()
    high_low = high - low
    print ('High:    %.2f\nLow:  %.2f\nHigh_low:  %.2f'%(high,low,high_low))
    opening = dt_stock['Price'].iloc[0]
    closing = dt_stock['Price'].iloc[-1]
    change = closing - opening
    pre_change = (change/opening)
    turnover = dt_stock['Quantity'].sum()
    print ('Opening:  {:.2f}\nClosing:  {:.2f}\nChange:  {:.2f}\npre_Change:  {:.2%}'.format(opening,closing,change,pre_change))
#     print ('\n\n\n')
    data = pd.DataFrame({"Date": date,'high-low':high_low,'change':change,'pre_change':pre_change,'turnover':turnover,'next_day_change':next_day_change},index=[0])
    dt_summary = pd.concat([dt_summary,data])
    return (dt_summary)

In [36]:
columns = ['Date', 'high-low', 'turnover', 'change','pre_change','next_day_change']
df = pd.DataFrame(columns=columns)
for i in range(len(date)):
    if i == len(date):break
    day = date[i]
    next_day = date[i+1]
    df = splitting_dt(df,dt_0005,day,next_day)

High:    58.35
Low:  57.85
High_low:  0.50
Opening:  58.15
Closing:  58.05
Change:  -0.10
pre_Change:  -0.17%
High:    58.90
Low:  58.15
High_low:  0.75
Opening:  58.30
Closing:  58.85
Change:  0.55
pre_Change:  0.94%
High:    59.40
Low:  58.60
High_low:  0.80
Opening:  58.75
Closing:  59.00
Change:  0.25
pre_Change:  0.43%
High:    59.55
Low:  58.95
High_low:  0.60
Opening:  59.30
Closing:  59.30
Change:  0.00
pre_Change:  0.00%
High:    59.05
Low:  58.45
High_low:  0.60
Opening:  59.00
Closing:  58.85
Change:  -0.15
pre_Change:  -0.25%
High:    59.80
Low:  58.50
High_low:  1.30
Opening:  59.50
Closing:  58.80
Change:  -0.70
pre_Change:  -1.18%
High:    58.95
Low:  58.45
High_low:  0.50
Opening:  58.55
Closing:  58.55
Change:  0.00
pre_Change:  0.00%
High:    58.55
Low:  57.35
High_low:  1.20
Opening:  58.55
Closing:  57.55
Change:  -1.00
pre_Change:  -1.71%
High:    58.60
Low:  57.90
High_low:  0.70
Opening:  57.95
Closing:  58.45
Change:  0.50
pre_Change:  0.86%
High:    58.35
Low: 

IndexError: list index out of range

In [37]:
bin_next_day = []
for change in df['next_day_change']:
    if change > 0:
        bin_next_day.append(1)
    else:
        bin_next_day.append(0)
df['bin_next_day'] = bin_next_day
print (df.head())

         Date  change  high-low  next_day_change  pre_change    turnover  \
0  2016-10-03   -0.10      0.50            -5.70   -0.001720  15855600.0   
0  2016-10-04    0.55      0.75            -5.25    0.009434  18135200.0   
0  2016-10-05    0.25      0.80            -4.70    0.004255  14070000.0   
0  2016-10-06    0.00      0.60            -5.00    0.000000  21178000.0   
0  2016-10-07   -0.15      0.60            -4.50   -0.002542  15188800.0   

   bin_next_day  
0             0  
0             0  
0             0  
0             0  
0             0  


*** Machine learning ***

In [44]:
from sklearn.cross_validation import train_test_split
def spliting_data(df_summary,ratio):
    df = df_summary.copy()
    # X_all, Y_all
    useless = ['Date','next_day_change']
    for column in useless:
        del df[column]
    X_all = df[[column for column in df.columns.tolist() if column != 'bin_next_day']]
    Y_all = df[['bin_next_day']]
    #####
    # set: Set the number of training points, Set the number of testing points
    num_train = int(len(X_all) * ratio)
    num_test = X_all.shape[0] - num_train
    #####
    X_train, X_test, y_train, y_test = train_test_split(X_all, Y_all, test_size=num_test, train_size=num_train, random_state=1)
    print ("Training set has {} samples.".format(X_train.shape[0]))
    print ("Testing set has {} samples.".format(X_test.shape[0]))
    return X_train, X_test, y_train, y_test

In [45]:
X_train, X_test, y_train, y_test = spliting_data(df,0.8)

Training set has 52 samples.
Testing set has 14 samples.


In [87]:
from time import time
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return (classification_report(target, y_pred))

def train_predict(clf, X_train, y_train, X_test, y_test):
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    print ('in-sample result:')
    print (predict_labels(clf, X_train, y_train))
    print ('out-sample result:')
    print (predict_labels(clf, X_test, y_test))

In [88]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=1)
clf_B = SVC(random_state=1)
clf_C = GaussianNB()

for clf in [clf_A,clf_B,clf_C]:
    train_predict(clf, X_train, y_train, X_test, y_test)
    print ('\n')

Training a DecisionTreeClassifier using a training set size of 52. . .
Trained model in 0.0009 seconds
in-sample result:
Made predictions in 0.0003 seconds.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        48
          1       1.00      1.00      1.00         4

avg / total       1.00      1.00      1.00        52

out-sample result:
Made predictions in 0.0002 seconds.
             precision    recall  f1-score   support

          0       0.71      1.00      0.83        10
          1       0.00      0.00      0.00         4

avg / total       0.51      0.71      0.60        14



Training a SVC using a training set size of 52. . .
Trained model in 0.0011 seconds
in-sample result:
Made predictions in 0.0004 seconds.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        48
          1       1.00      1.00      1.00         4

avg / total       1.00      1.00      1.00        52

out

/Users/admin/anaconda2/envs/python3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/admin/anaconda2/envs/python3/lib/python3.5/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/Users/admin/anaconda2/envs/python3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
